In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import sys
from scipy import sparse
import seaborn as sns
import matplotlib.pyplot as plt
sys.path.append("../utils/")
import pre_processing
import sci_excerpts
import anndata2ri
import rpy2.robjects as ro
import rpy2.rinterface_lib.callbacks

In [2]:
adata = sc.read_h5ad("../../process/pre-intergration/big_data/20241019_mergeall_filter_gene_step1.h5ad")

In [4]:
n_pcs=50
counts_per_cell = 1e4
louvain_r=0.5
log_transform=False
anndata2ri.activate()
ro.r('library("scran")')
    
    # keep raw counts
adata.layers["counts"] = adata.X.copy()
    
    # Preliminary clustering for differentiated normalisation
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp, counts_per_cell_after=counts_per_cell)
sc.pp.log1p(adata_pp)
sc.pp.pca(adata_pp, n_comps=n_pcs, svd_solver='arpack')
sc.pp.neighbors(adata_pp)


/tmp/ipykernel_2100804/949712234.py:5: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, c

ModuleNotFoundError: No module named 'louvain'

In [5]:
sc.tl.louvain(adata_pp, key_added='groups', resolution=louvain_r)  

ro.globalenv['data_mat'] = adata.X.T
ro.globalenv['input_groups'] = adata_pp.obs['groups']
size_factors = ro.r('sizeFactors(computeSumFactors(SingleCellExperiment('
                            'list(counts=data_mat)), clusters = input_groups,'
                            f' min.mean = {0.1}))')

R[write to console]: Error in (function (cond)  : 
  error in evaluating the argument 'object' in selecting a method for function 'sizeFactors': BiocParallel errors
  1 remote errors, element index: 1
  149 unevaluated and other errors
  first remote error:
Error in as(x, "dgCMatrix"): no method or default for coercing “dgRMatrix” to “dgCMatrix”

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



RRuntimeError: Error in (function (cond)  : 
  error in evaluating the argument 'object' in selecting a method for function 'sizeFactors': BiocParallel errors
  1 remote errors, element index: 1
  149 unevaluated and other errors
  first remote error:
Error in as(x, "dgCMatrix"): no method or default for coercing “dgRMatrix” to “dgCMatrix”


In [6]:
ro.r('data_mat <- as(data_mat, "CsparseMatrix")')
size_factors = ro.r('sizeFactors(computeSumFactors(SingleCellExperiment('
                            'list(counts=data_mat)), clusters = input_groups,'
                            f' min.mean = {0.1}))')

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (x, sizes, min.mean = NULL, positive = FALSE, scaling = NULL)  :
R[write to console]: 
 
R[write to console]:  encountered non-positive size factor estimates

R[write to console]: 2: 
R[write to console]: In (function (x, sizes, min.mean = NULL, positive = FALSE, scaling = NULL)  :
R[write to console]: 
 
R[write to console]:  encountered non-positive size factor estimates

R[write to console]: 3: 
R[write to console]: In (function (x, sizes, min.mean = NULL, positive = FALSE, scaling = NULL)  :
R[write to console]: 
 
R[write to console]:  encountered non-positive size factor estimates

R[write to console]: 4: 
R[write to console]: In (function (x, sizes, min.mean = NULL, positive = FALSE, scaling = NULL)  :
R[write to console]: 
 
R[write to console]:  encountered non-positive size factor estimates



In [7]:
sizeTb = pd.DataFrame(size_factors)
sizeTb.to_csv("../../process/pre-intergration/table/20241019_size_factor.csv")

In [9]:
sizeTb

0
0       0.339254
1       0.217982
2       1.260583
3       0.263368
4       0.857411
...          ...
371462  0.897740
371463  0.768228
371464  0.640589
371465  1.040026
371466  0.875229

[371467 rows x 1 columns]

In [10]:
adata.obs['size_factors'] = sizeTb[0].values
# modify adata
adata.X /= adata.obs['size_factors'].values[:,None]

    # convert to sparse, bc operation always converts to dense
adata.X = sparse.csr_matrix(adata.X)

In [11]:
adata.layers["counts"] = adata.X

In [12]:
adata.write_h5ad("../../process/pre-intergration/big_data/20241019_mergeall_normalized_step2.h5ad")

In [13]:
adata.obs["Sample"] = adata.obs['orig.ident'].copy()

In [14]:
adata.obs.to_csv("../../processed_data/metadata/20241019_metadata.csv")

In [17]:
adata.obs

orig.ident  nCount_RNA  \
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1   ToothNc_Junjun_E13.5      4461.0   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1   ToothNc_Junjun_E13.5      2587.0   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1   ToothNc_Junjun_E13.5     14429.0   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1   ToothNc_Junjun_E13.5      2657.0   
ToothNc_Junjun_AAACCCACACAACGAG-1_1   ToothNc_Junjun_E13.5     10948.0   
...                                                    ...         ...   
MolarP12_Tomoko_TTTGGTTGTTGTCAGT-1_2  MolarP12_Tomoko_homo      9761.0   
MolarP12_Tomoko_TTTGGTTTCCGTTTCG-1_2  MolarP12_Tomoko_homo      8938.0   
MolarP12_Tomoko_TTTGGTTTCTAACACG-1_2  MolarP12_Tomoko_homo      6486.0   
MolarP12_Tomoko_TTTGTTGAGAGGTATT-1_2  MolarP12_Tomoko_homo     12096.0   
MolarP12_Tomoko_TTTGTTGTCTTACCGC-1_2  MolarP12_Tomoko_homo     15565.0   

                                      nFeature_RNA coarse_anno_1  \
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1           1712    Epithelium   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1           1233    Mesenchyme   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1           3934    Epithelium   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1           1418           RBC   
ToothNc_Junjun_AAACCCACACAACGAG-1_1           3213   Endothelium   
...                                            ...           ...   
MolarP12_Tomoko_TTTGGTTGTTGTCAGT-1_2          3318  Perivascular   
MolarP12_Tomoko_TTTGGTTTCCGTTTCG-1_2          3105  Perivascular   
MolarP12_Tomoko_TTTGGTTTCTAACACG-1_2          2708        Neuron   
MolarP12_Tomoko_TTTGTTGAGAGGTATT-1_2          3596        Neuron   
MolarP12_Tomoko_TTTGTTGTCTTACCGC-1_2          2956        Immune   

                                      total_counts  log10_total_counts  \
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1         4461.0            3.649432   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1         2587.0            3.412796   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1        14429.0            4.159236   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1         2657.0            3.424392   
ToothNc_Junjun_AAACCCACACAACGAG-1_1        10948.0            4.039335   
...                                            ...                 ...   
MolarP12_Tomoko_TTTGGTTGTTGTCAGT-1_2        9761.0            3.989494   
MolarP12_Tomoko_TTTGGTTTCCGTTTCG-1_2        8938.0            3.951240   
MolarP12_Tomoko_TTTGGTTTCTAACACG-1_2        6486.0            3.811977   
MolarP12_Tomoko_TTTGTTGAGAGGTATT-1_2       12096.0            4.082642   
MolarP12_Tomoko_TTTGTTGTCTTACCGC-1_2       15565.0            4.192149   

                                      n_genes_detected  mito_frac  ribo_frac  \
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1               1712   0.071509   0.298364   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1               1233   0.143796   0.187089   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1               3934   0.045533   0.231478   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1               1418   0.062476   0.212269   
ToothNc_Junjun_AAACCCACACAACGAG-1_1               3213   0.074351   0.272653   
...                                                ...        ...        ...   
MolarP12_Tomoko_TTTGGTTGTTGTCAGT-1_2              3318   0.052351   0.169655   
MolarP12_Tomoko_TTTGGTTTCCGTTTCG-1_2              3105   0.033229   0.092638   
MolarP12_Tomoko_TTTGGTTTCTAACACG-1_2              2708   0.049337   0.103299   
MolarP12_Tomoko_TTTGTTGAGAGGTATT-1_2              3596   0.068287   0.085565   
MolarP12_Tomoko_TTTGTTGTCTTACCGC-1_2              2956   0.010215   0.307613   

                                         compl  size_factors  \
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1   0.383770      0.339254   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1   0.476614      0.217982   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1   0.272645      1.260583   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1   0.533685      0.263368   
ToothNc_Junjun_AAACCCACACAACGAG-1_1   0.293478      0.857411   
...                                        ...           ...   
MolarP12_Tomoko_TTTGGTTGTTGTCAGT-1_2  0.3399

In [15]:
!cp ../../processed_data/metadata/20241019_metadata.csv ../../processed_data/metadata/scMetadata_latest.csv

In [18]:
!python ../utils/update_metadata.py

writing to ../../processed_data/metadata/scMetadata_latest.csv


In [19]:
scMeta = pd.read_csv("../../processed_data/metadata/scMetadata_latest.csv",index_col=0)

/tmp/ipykernel_2100804/3350141961.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  scMeta = pd.read_csv("../../processed_data/metadata/scMetadata_latest.csv",index_col=0)


In [22]:
scMeta.head()

orig.ident  nCount_RNA  \
index                                                                   
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1  ToothNc_Junjun_E13.5      4461.0   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1  ToothNc_Junjun_E13.5      2587.0   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1  ToothNc_Junjun_E13.5     14429.0   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1  ToothNc_Junjun_E13.5      2657.0   
ToothNc_Junjun_AAACCCACACAACGAG-1_1  ToothNc_Junjun_E13.5     10948.0   

                                     nFeature_RNA coarse_anno_1  total_counts  \
index                                                                           
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1          1712    Epithelium        4461.0   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1          1233    Mesenchyme        2587.0   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1          3934    Epithelium       14429.0   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1          1418           RBC        2657.0   
ToothNc_Junjun_AAACCCACACAACGAG-1_1          3213   Endothelium       10948.0   

                                     log10_total_counts  n_genes_detected  \
index                                                                       
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1            3.649432              1712   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1            3.412796              1233   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1            4.159236              3934   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1            3.424392              1418   
ToothNc_Junjun_AAACCCACACAACGAG-1_1            4.039335              3213   

                                     mito_frac  ribo_frac     compl  ...  \
index                                                                ...   
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1   0.071509   0.298364  0.383770  ...   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1   0.143796   0.187089  0.476614  ...   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1   0.045533   0.231478  0.272645  ...   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1   0.062476   0.212269  0.533685  ...   
ToothNc_Junjun_AAACCCACACAACGAG-1_1   0.074351   0.272653  0.293478  ...   

                                     Development stage       Disease   Strain  \
index                                                                           
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1                Bud  Not provided  C57BL/6   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1                Bud  Not provided  C57BL/6   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1                Bud  Not provided  C57BL/6   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1                Bud  Not provided  C57BL/6   
ToothNc_Junjun_AAACCCACACAACGAG-1_1                Bud  Not provided  C57BL/6   

                                                           Dissociation_enzyme  \
index                                                                            
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1  Dispase (Roche) and Collagenase I (Gibco)   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1  Dispase (Roche) and Collagenase I (Gibco)   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1  Dispase (Roche) and Collagenase I (Gibco)   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1  Dispase (Roche) and Collagenase I (Gibco)   
ToothNc_Junjun_AAACCCACACAACGAG-1_1  Dispase (Roche) and Collagenase I (Gibco)   

                                          Machine       Species  \
index                                                             
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1  10X(unknown)  Mus Musculus   
ToothNc_Junjun_AAACCCAAGCTAAGTA-1_1  10X(unknown)  Mus Musculus   
ToothNc_Junjun_AAACCCAAGTGATTCC-1_1  10X(unknown)  Mus Musculus   
ToothNc_Junjun_AAACCCACAAGATTGA-1_1  10X(unknown)  Mus Musculus   
ToothNc_Junjun_AAACCCACACAACGAG-1_1  10X(unknown)  Mus Musculus   

                                                                         Related assay  \
index                                                                                    
ToothNc_Junjun_AAACCCAAGCCTGTCG-1_1  Spatiotemporal single-cell regulatory atlas re...   
ToothNc_Jun

In [23]:
adata.obs =  scMeta

In [25]:
set(adata.obs["Project"])

{'Atlas_Jan',
 'CAGE_Chiba',
 'Cycling_Rapolas',
 'Epi_Chiba',
 'Incisor_Luo',
 'Incisor_Zhang',
 'LYPD1_Chiba',
 'MolarE145_Outie',
 'MolarP12_Tomoko',
 'Molar_Qian',
 'Molar_Raju',
 'Peridontal_Nagata',
 'Runx2_Shuo',
 'ToothNc_Junjun',
 'ToothNiche_Hong',
 'Tooth_Zheng',
 nan}

In [28]:
adata.obs.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'coarse_anno_1',
       'total_counts', 'log10_total_counts', 'n_genes_detected', 'mito_frac',
       'ribo_frac', 'compl', 'size_factors', 'Sample', 'Project',
       'Core_datasets', 'Mandibular_Maxillary', 'Molar_Incisor',
       'Tooth position', 'Histology', 'Sex', 'Gene Type', 'Knockout_gene',
       'Cre', 'Treatment', 'FACs', 'Age', 'Stage', 'Development stage',
       'Disease', 'Strain', 'Dissociation_enzyme', 'Machine', 'Species',
       'Related assay', 'Journal', 'Data Source', 'Data location'],
      dtype='object')

In [32]:
anno = pd.read_csv("../../process/annotation/first_round_base/anno/Atlas_Jan_Mouse.csv",index_col=0)

In [37]:
anno["Coarse_Label_1"].values

array(['Mesenchyme', 'Mesenchyme', 'Mesenchyme', ..., 'perivascular',
       'Endothelium', 'perivascular'], dtype=object)

In [38]:
adata.obs["coarse_anno_1"][adata.obs["Project"]=="Atlas_Jan"] = anno["Coarse_Label_1"].values

/tmp/ipykernel_2100804/992035526.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  adata.obs["coarse_anno_1"][adata.obs["Project"]=="Atlas_Jan"] = anno["Coarse_Label_1"].values
/tmp/ipykernel_2100804/992035526.py:1: SettingWithCopyWarning:

In [39]:
adata.obs["coarse_anno_1"][adata.obs["Project"]=="Atlas_Jan"] 

index
Atlas_Jan_Mouse_renamed_5_one_AAACCTGAGAGAGCTC-1      Mesenchyme
Atlas_Jan_Mouse_renamed_5_one_AAACCTGAGGAGTAGA-1      Mesenchyme
Atlas_Jan_Mouse_renamed_5_one_AAACCTGCACGTGAGA-1      Mesenchyme
Atlas_Jan_Mouse_renamed_5_one_AAACCTGGTCGCTTTC-1      Mesenchyme
Atlas_Jan_Mouse_renamed_5_one_AAACCTGGTTAAGTAG-1          Immune
                                                        ...     
Atlas_Jan_Mouse_renamed_one_TTTGACTTCTAGACCA-1      perivascular
Atlas_Jan_Mouse_renamed_one_TTTGATCGTCCACACG-1        Mesenchyme
Atlas_Jan_Mouse_renamed_one_TTTGATCTCCCGATCT-1      perivascular
Atlas_Jan_Mouse_renamed_one_TTTGGAGTCGACTCCT-1       Endothelium
Atlas_Jan_Mouse_renamed_one_TTTGGAGTCTTCCACG-1      perivascular
Name: coarse_anno_1, Length: 9164, dtype: object

In [41]:
adata.obs.to_csv("../../processed_data/metadata/20241019_metadata.csv")

In [44]:
adata.obs["Core_datasets"] = adata.obs["Core_datasets"].astype(str)

In [42]:
!cp ../../processed_data/metadata/20241019_metadata.csv ../../processed_data/metadata/scMetadata_latest.csv

In [45]:
adata.write_h5ad("../../process/pre-intergration/big_data/20241019_mergeall_normalized_step2.h5ad")

In [49]:
adata.obs.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'coarse_anno_1',
       'total_counts', 'log10_total_counts', 'n_genes_detected', 'mito_frac',
       'ribo_frac', 'compl', 'size_factors', 'Sample', 'Project',
       'Core_datasets', 'Mandibular_Maxillary', 'Molar_Incisor',
       'Tooth position', 'Histology', 'Sex', 'Gene Type', 'Knockout_gene',
       'Cre', 'Treatment', 'FACs', 'Age', 'Stage', 'Development stage',
       'Disease', 'Strain', 'Dissociation_enzyme', 'Machine', 'Species',
       'Related assay', 'Journal', 'Data Source', 'Data location'],
      dtype='object')

In [56]:
adata.obs[adata.obs["Project"].isnull()]["orig.ident"].unique()

['Cycling_Rapolas_injury', 'Follicle_Akira']
Categories (40, object): ['Atlas_Jan_incisor_10x', 'Atlas_Jan_incisor_SS2', 'Atlas_Jan_molar_10x.1', 'Atlas_Jan_molar_10x.2', ..., 'ToothNiche_Hong_PN1', 'ToothNiche_Hong_PN7', 'Tooth_Zheng_M1', 'Tooth_Zheng_M2']

In [57]:
!python ../utils/update_metadata.py

/home/zhanglab/202409_tooth/script/prepare_integration/../utils/update_metadata.py:9: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  scMeta = pd.read_csv(scMeta_path, index_col=0)
writing to ../../processed_data/metadata/scMetadata_latest.csv


In [58]:
scMeta = pd.read_csv("../../processed_data/metadata/scMetadata_latest.csv",index_col=0)

/tmp/ipykernel_2100804/3350141961.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  scMeta = pd.read_csv("../../processed_data/metadata/scMetadata_latest.csv",index_col=0)


In [68]:
adata.obs.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'coarse_anno_1',
       'total_counts', 'log10_total_counts', 'n_genes_detected', 'mito_frac',
       'ribo_frac', 'compl', 'size_factors', 'Sample', 'Project',
       'Core_datasets', 'Mandibular_Maxillary', 'Molar_Incisor',
       'Tooth position', 'Histology', 'Sex', 'Gene Type', 'Knockout_gene',
       'Cre', 'Treatment', 'FACs', 'Age', 'Stage', 'Development stage',
       'Disease', 'Strain', 'Dissociation_enzyme', 'Machine', 'Species',
       'Related assay', 'Journal', 'Data Source', 'Data location'],
      dtype='object')

In [70]:
# Iterate over the columns in the DataFrame
for column in scMeta.columns:
    if column.endswith('_x'):
        # Construct the corresponding '_y' column name
        y_column = column[:-2] + '_y'
        
        # Check if the '_y' column exists in the DataFrame
        if y_column in scMeta.columns:
            # Update the '_x' column with values from the '_y' column
            scMeta[column].update(scMeta[y_column])

# Optionally, drop the '_y' columns if they are no longer needed
scMeta.drop(columns=[col for col in scMeta.columns if col.endswith('_y')], inplace=True)

# Rename the '_x' columns to remove the suffix
scMeta.columns = [col.replace('_x', '') for col in scMeta.columns]

/tmp/ipykernel_2100804/4016639810.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  scMeta[column].update(scMeta[y_column])


In [73]:
set(scMeta["Project"])

{'Atlas_Jan',
 'CAGE_Chiba',
 'Cycling_Rapolas',
 'Epi_Chiba',
 'Follicle_Akira',
 'Incisor_Luo',
 'Incisor_Zhang',
 'LYPD1_Chiba',
 'MolarE145_Outie',
 'MolarP12_Tomoko',
 'Molar_Qian',
 'Molar_Raju',
 'Peridontal_Nagata',
 'Runx2_Shuo',
 'ToothNc_Junjun',
 'ToothNiche_Hong',
 'Tooth_Zheng'}

In [74]:
scMeta.to_csv("../../processed_data/metadata/scMetadata_latest.csv")